In [119]:
import pandas as pd
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings


pd.options.mode.chained_assignment = None


# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ
conn_str_local = 'DRIVER={SQL Server};SERVER='+server_local+';DATABASE='+database_local+';'

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()
# формируем заголовки для дальнейших запросов
headers = {
    'Content-Type': 'application/json', 
    'Accept': 'application/json', 
    'Authorization': f'OAuth {yandex_token}'
}

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'

# УРЛ для операций над опубликованными папками и файлами на Яндекс Диске
base_public_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?'  
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' # УРЛ для скачивания эксель файлов из Яндекс Диска
delete_url = 'https://cloud-api.yandex.net/v1/disk/resources?' # УРЛ для удаления папок и файлов на Яндекс Диске

final_url = base_public_url + urlencode(dict(public_key=public_key))  # Формируем УРЛ для получения списка папок и файлов
res = requests.get(final_url)  # Отправлем запрос на Яндекс диск, чтобы получить название папок и public key для скачивания файлов

print(res.status_code)

yandex_folders = res.json() # парсим ответ

200


In [120]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [121]:
# Создаем функцию для получения содержимого Яндекс диска
#  на входе принимаем 
# - адрес запроса для скачивания файла
# - public key, который мы получаем из ответа Яндекс при запросе информации по конкретной папке
# - адрес папки
# - лимит файлов (максимальное кол-во 80)
def get_yandex_disk_responce(request_url, public_key, folder_path, limit=80):
    res_url = request_url + urlencode(dict(public_key=public_key, path=folder_path, limit=limit)) # формируем строку запроса
    res = requests.get(res_url) # отправляем запрос на сервер
    yandex_responce = res.json() # получаем ответ и преобразуем его в json

    return yandex_responce

In [122]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_new_sources_files
/02_regular_reports
one_target_video


In [123]:
data_link = download_response.content

In [124]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [153]:
sheet_names.sheet_names

['Отчет по всем РК', 'Отчет по дням', 'Worksheet 1']

In [126]:
report_type = 'test'

In [170]:
tmp_video_dict = {}

df = pd.read_excel(BytesIO(data_link), sheet_name='Отчет по дням', header=None)
df = df.fillna('')

# каждый продукт имеет свой диапазон строк
# диапазон может отличаться (какие-то продукты запускаются раньше, какие-то позже)
start_index_list = df[df[0]=='Дата'].index # забираем индекс строки для отсчета начала диапазона
end_index_list = df[df[0]=='Итог'].index # забираем индекс строки для окончания диапазона

df.columns = df.iloc[start_index_list[0]].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
# привоодим названия к единому стандарту
df = df.rename(columns={'дата': 'date', 'показы': 'impressions', 'клики': 'clicks', 'охват': 'reach'}) 
df = df[['date', 'impressions','reach', 'clicks']] # оставляем только нужные поля

# теперь сформируем датаФрейм для каждого отдельного продукта
# через цикл перебираем список индексов начала диапазона
for i in range(len(start_index_list)):
    start_index = start_index_list[i] # берем индекс начала
    end_index = end_index_list[i] # берем индекс окончания

    # название продукта находится сверху таблицы с данными. поэтому нам нужна предыдущая ячейка перед начальным индексом
    product = df['date'][start_index-1] # забираем название продукта
    # print(index_product_list[i])
    df_tmp = df.iloc[start_index+1:end_index] # забираем строки из диапазона

    df_tmp['source'] = 'one target'
    df_tmp['format_type'] = 'video'
    df_tmp['product'] =  product
    df_tmp['date'] = pd.to_datetime(df_tmp['date']).dt.date # приводим в формат даты
    # df_tmp = df_tmp[df_tmp['impressions']!=0] # убираем дни, в которых не было показов объявлений
    df_tmp['product'] =  df_tmp['product'].str.lower().str.strip()
    # сохраняем датаФрейм во временный словарь 
    # ключ - это название продукта (15s, 6s и тд)
    df_tmp['report_type'] = report_type # сохраняем в отдельном поле - относится отчет к бонусным или нет
    tmp_video_dict[product] = df_tmp

In [171]:
tmp_video_dict

{'15Snew': 2        date impressions         reach clicks      source format_type  \
 3  2024-07-28       48993         35975    380  one target       video   
 4  2024-07-29       46157  24165.968586    381  one target       video   
 5  2024-07-30       29636  15355.440415    453  one target       video   
 6  2024-07-31       24932  12985.416667    440  one target       video   
 
 2 product report_type  
 3  15snew        test  
 4  15snew        test  
 5  15snew        test  
 6  15snew        test  ,
 '6snew': 2         date impressions         reach clicks      source format_type  \
 11  2024-07-28       48615         35697    300  one target       video   
 12  2024-07-29       37744  19062.626263    280  one target       video   
 13  2024-07-30       28745  14517.676768    300  one target       video   
 14  2024-07-31       27439  14071.282051     54  one target       video   
 
 2  product report_type  
 11   6snew        test  
 12   6snew        test  
 13   6snew       

In [ ]:
# источник One target
# тип отчета Видео реклама
# Функция для обработки статистики по дням для обычных и бонусных размещений 
# Статистика по всем креативам на 2-м листе
# Каждый креатив в отдельной таблице
# Таблицы располагаются друг над другом
def get_one_target_video_base_bonus_report(data_link, report_type):
    tmp_video_dict = {}
    
    cols_range = 'A:E' # задаем диапазон полей, которые нам нужны
    # чтобы не сохранять файл на комп, мы пробрасываем его сразу в Pandas
    one_target_df = pd.read_excel(BytesIO(data_link), sheet_name='Отчет по дням', header=None, usecols=cols_range,
                                  names=['date', 'impressions', 'clicks', 'CTR %', 'reach'])
    one_target_df = one_target_df[['date', 'impressions', 'clicks', 'reach']] # убриаем лишние поля

    # каждый продукт имеет свой диапазон строк
    # диапазон может отличаться (какие-то продукты запускаются раньше, какие-то позже)
    index_product_list = one_target_df[one_target_df['date']=='Дата'].index # забираем индекс строки для отсчета начала диапазона
    total_row_list = one_target_df[one_target_df['date']=='Итог'].index # забираем индекс строки для окончания диапазона

    # теперь сформируем датаФрейм для каждого отдельного продукта
    # через цикл перебираем список индексов начала диапазона
    for i in range(len(index_product_list)):
        start_index = index_product_list[i] # берем индекс начала
        end_index = total_row_list[i] # берем индекс окончания
        # название продукта находится сверху таблицы с данными. поэтому нам нужна предыдущая ячейка перед начальным индексом
        df_name = one_target_df['date'][start_index-1] # забираем название продукта
        # print(index_product_list[i])
        df = one_target_df.iloc[start_index+1:end_index] # забираем строки из диапазона

        df['source'] = 'one target'
        df['format_type'] = 'video'
        df.insert(1, 'product', df_name, True) # добавляем статичное поле с названием продукта
        df['date'] = pd.to_datetime(df['date']).dt.date # приводим в формат даты
        df['product'] = df['product'].str.lower()
        df = df[df['impressions']!=0] # убираем дни, в которых не было показов объявлений
        # сохраняем датаФрейм во временный словарь 
        # ключ - это название продукта (15s, 6s и тд)
        df['report_type'] = report_type # сохраняем в отдельном поле - относится отчет к бонусным или нет
        tmp_video_dict[df_name] = df
        
    return pd.concat(tmp_video_dict, ignore_index=True)